<a href="https://colab.research.google.com/github/drewwint/cog_ctrl_comp_sssst/blob/main/sssst_simulation_and_model_build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

simulating dataset

In [ ]:
def cog_cap(dd):
  cc = ((
    ((np.max(dd.ssd) - dd.ssd)/np.max(dd.ssd)) *
    np.exp(-((dd.ssd/np.max(data.ssd)) ** np.log(dd.timeseries+1)))
    ) * np.log1p(dd.accuracy + 0.5))

  return cc

In [ ]:
def cog_cap(dd):
  inh_cap = ((np.max(dd.ssd) - dd.ssd)/np.max(dd.ssd))

  cc = ((
    ((np.max(dd.ssd) - dd.ssd)/np.max(dd.ssd)) *
    np.exp(-((dd.ssd/np.max(data.ssd)) ** np.log(dd.timeseries+1)))
    ) * np.log1p(dd.accuracy + 0.5))

  return cc

In [ ]:
def cog_cap(dd):
  """
  Computes a cognitive capacity score based on trial-level data.

  This function calculates a cognitive capacity score using a custom formula
  that incorporates stop-signal delay (SSD), trial number (timeseries),
  and accuracy. The formula is likely based on assumptions about the
  relationship between these variables and cognitive control processes.

  Args:
      dd (pd.DataFrame): DataFrame containing trial-level data with columns:
                         - 'ssd': Stop-signal delay.
                         - 'timeseries': Trial number or a time-dependent variable.
                         - 'accuracy': Accuracy on the trial (0 or 1).

  Returns:
      float: Cognitive capacity score.
  """
  cc = (
      (  # Start of the main calculation
          (
              (np.max(dd.ssd) - dd.ssd)  # Difference between max SSD and current trial's SSD
              / np.max(dd.ssd)  # Normalize by max SSD
          )
          * np.exp(  # Multiply by an exponential decay factor
              -(  # Negate the exponent
                  (
                      dd.ssd / np.max(data.ssd)  # Ratio of current SSD to max SSD (from external data)
                  )
                  ** np.log(dd.timeseries + 1)  # Exponent based on trial number (timeseries)
              )
          )
      )
      * np.log1p(dd.accuracy + 0.5)  # Multiply by log-transformed accuracy (smoothed)
  )
  return cc  # Return the calculated cognitive capacity score

## Function to Simulate SSSST data
Note we have to input values via a dictionary for
1. probability and variance of being accurate
2. reaction time (rt) by accuracy and trial

In [59]:
import numpy as np
import pandas as pd

def simulate_reaction_times(n_trials, params, rt_params, seed=42):
    # Seed for reproducibility
    np.random.seed(seed)

    # Define SSD (Stop Signal Delay) constraints
    ssd_start = 0.300
    ssd_min = 0.050
    ssd_max = 1.100
    ssd_increment = 0.050

    # Generate trial types in blocks of 5 trials with random order
    trial_type_blocks = []
    for _ in range(n_trials // 5):
        block = np.random.permutation([1, 1, 1, 0, 2])
        trial_type_blocks.extend(block)

    # Trim or pad if n_trials is not divisible by 5
    trial_type = trial_type_blocks[:n_trials]

    # Initialize lists for data storage
    reaction_times = []
    stop_signal_delays = []
    accuracies = []

    # Initial SSD
    ssd = ssd_start

    for i, trial in enumerate(trial_type):
        # Accuracy
        acc = np.random.binomial(1,
                                 np.clip(np.random.normal(params[trial]['accuracy']['mean'],
                                                          params[trial]['accuracy']['std']), 0, 1))
        accuracies.append(acc)

        # Reaction Time based on trial type and accuracy
        rt = np.random.normal(rt_params[(trial, acc)]['mean'], rt_params[(trial, acc)]['std'])
        rt = np.clip(rt, 0, 1.5)  # Ensure reaction time is between 0 and 1.5 seconds
        reaction_times.append(rt)

        # Stop Signal Delay adjustment for the next trial if current trial is type 2
        if i > 0 and trial_type[i-1] == 2:  # Check if the previous trial was type 2
            if accuracies[i-1] == 1:
                ssd = min(ssd + ssd_increment, ssd_max)  # Increase by increment if previous trial #2 was correct
            else:
                ssd = max(ssd - ssd_increment, ssd_min)  # Decrease by increment if previous trial #2 was incorrect

        stop_signal_delays.append(ssd)

    # Create DataFrame
    data = pd.DataFrame({
        'trial_type': trial_type,
        'reaction_time': reaction_times,
        'stop_signal_delay': stop_signal_delays,
        'accuracy': accuracies
    })

    # Optionally, check statistics by trial type
    for t in [0, 1, 2]:
        print(f"\nStatistics for Trial Type {t}:")
        print(data[data['trial_type'] == t].describe())
        print(f"\nAverage reaction time by accuracy for Trial Type {t}:", "\n", pd.DataFrame({"accuracy":[0,1],"mean_rt":data[data['trial_type'] == t].groupby("accuracy").reaction_time.mean().values}),"\n")

    return data



## Simulating data for subject in the test condition
We obtained values from the real data

In [67]:

params = {
    0: {'accuracy': {'mean': 0.675, 'std': 0.501}},
    1: {'accuracy': {'mean': 0.899, 'std': 0.348}},
    2: {'accuracy': {'mean': 0.335, 'std': 0.4813}}
}

rt_params = {
    (0, 0): {'mean': 0.510007, 'std': 0.236518},
    (0, 1): {'mean': 1.500, 'std': 0},
    (1, 0): {'mean': 1.461129, 'std': 0.239617},
    (1, 1): {'mean': 0.743676, 'std': 0.272865},
    (2, 0): {'mean': 0.608010, 'std': 0.486115},
    (2, 1): {'mean': 0.533259, 'std': 0.154071}
}

test_df = simulate_reaction_times(450, params, rt_params)
print("\n", "dataframe head", "\n", test_df.head())


Statistics for Trial Type 0:
       trial_type  reaction_time  stop_signal_delay   accuracy
count        90.0      90.000000          90.000000  90.000000
mean          0.0       1.060139           0.220000   0.533333
std           0.0       0.504849           0.132351   0.501683
min           0.0       0.000000           0.050000   0.000000
25%           0.0       0.562750           0.100000   0.000000
50%           0.0       1.500000           0.200000   1.000000
75%           0.0       1.500000           0.350000   1.000000
max           0.0       1.500000           0.450000   1.000000

Average reaction time by accuracy for Trial Type 0: 
    accuracy  mean_rt
0         0  0.55744
1         1  1.50000 


Statistics for Trial Type 1:
       trial_type  reaction_time  stop_signal_delay    accuracy
count       270.0     270.000000         270.000000  270.000000
mean          1.0       0.873580           0.220741    0.814815
std           0.0       0.357965           0.135773    0.3891

### Evaluating stimulation

In [68]:
test_df

,trial_type,reaction_time,stop_signal_delay,accuracy
0,1,0.769261,0.30,1
1,2,0.401196,0.30,1
2,1,0.706219,0.35,1
3,1,0.938502,0.35,1
4,0,0.685111,0.35,0
...,...,...,...,...
445,0,0.450680,0.35,0
446,1,1.077834,0.35,1
447,1,1.500000,0.35,0
448,2,0.810475,0.35,1


In [69]:
test_df.stop_signal_delay.value_counts()


,count
stop_signal_delay,
0.05,76
0.35,60
0.30,59
0.40,48
0.20,34
0.25,33
0.10,33
0.10,32
0.15,29


In [12]:
import numpy as np
import pandas as pd

# Seed for reproducibility
np.random.seed(42)

n_trials = 450

# Define parameters for each trial type
params = {
    0: {
        'accuracy': {'mean': 0.533, 'std': 0.501}
    },
    1: {
        'accuracy': {'mean': 0.859, 'std': 0.348}
    },
    2: {
        'accuracy': {'mean': 0.3555, 'std': 0.4813}
    }
}

# Define SSD (Stop Signal Delay) constraints
ssd_start = 0.300
ssd_min = 0.050
ssd_max = 1.100
ssd_increment = 0.050

# Generate trial types in blocks of 5 trials with random order
trial_type_blocks = []
for _ in range(n_trials // 5):
    block = np.random.permutation([1, 1, 1, 0, 2])
    trial_type_blocks.extend(block)

# Trim or pad if n_trials is not divisible by 5
trial_type = trial_type_blocks[:n_trials]

# Initialize lists for data storage
reaction_times = []
stop_signal_delays = []
accuracies = []

# Initial SSD
ssd = ssd_start

for i, trial in enumerate(trial_type):
    # Accuracy
    acc = np.random.binomial(1,
                             np.clip(np.random.normal(params[trial]['accuracy']['mean'],
                                                      params[trial]['accuracy']['std']), 0, 1))
    accuracies.append(acc)

    # Reaction Time (timeseries) based on new rules
    if trial == 0:
        if acc == 1:
            rt = 1.25
        else:
            rt = np.random.uniform(0, 1.25)
    else:  # trial is 1 or 2
        if acc == 1:
            rt = np.random.uniform(0, 1.25)
        else:
            rt = 1.25
    reaction_times.append(rt)

    # Stop Signal Delay adjustment
    if trial == 2:
        if acc == 1:
            ssd = min(ssd + ssd_increment, ssd_max)  # Increase by increment, but cap at ssd_max
        else:
            ssd = max(ssd - ssd_increment, ssd_min)  # Decrease by increment, but not below ssd_min
    stop_signal_delays.append(ssd)

# Create DataFrame
data = pd.DataFrame({
    'trial_type': trial_type,
    'reaction_time': reaction_times,
    'stop_signal_delay': stop_signal_delays,
    'accuracy': accuracies
})

# Display first few rows to check data
print(data.head())

# Optionally, check statistics by trial type
for t in [0, 1, 2]:
    print(f"\nStatistics for Trial Type {t}:")
    print(data[data['trial_type'] == t].describe())

   trial_type  reaction_time  stop_signal_delay  accuracy
0           1       0.244054               0.30         1
1           2       0.350965               0.35         1
2           1       1.175573               0.35         1
3           1       1.250000               0.35         0
4           0       1.160398               0.35         0

Statistics for Trial Type 0:
       trial_type  reaction_time  stop_signal_delay   accuracy
count        90.0      90.000000          90.000000  90.000000
mean          0.0       0.928658           0.131667   0.488889
std           0.0       0.404868           0.079128   0.502677
min           0.0       0.022791           0.050000   0.000000
25%           0.0       0.599179           0.050000   0.000000
50%           0.0       1.213525           0.100000   0.000000
75%           0.0       1.250000           0.200000   1.000000
max           0.0       1.250000           0.350000   1.000000

Statistics for Trial Type 1:
       trial_type  reacti

In [15]:
import numpy as np
import pandas as pd

# Seed for reproducibility
np.random.seed(42)

n_trials = 450

# Define parameters for each trial type
params = {
    0: {
        'accuracy': {'mean': 0.533, 'std': 0.501},
    },
    1: {
        'accuracy': {'mean': 0.859, 'std': 0.348},
    },
    2: {
        'accuracy': {'mean': 0.3555, 'std': 0.4813},
    }
}

# Define SSD (Stop Signal Delay) constraints
ssd_start = 0.300
ssd_min = 0.050
ssd_max = 1.100
ssd_increment = 0.050

# Generate trial types in blocks of 5 trials with random order
trial_type_blocks = []
for _ in range(n_trials // 5):
    block = np.random.permutation([1, 1, 1, 0, 2])
    trial_type_blocks.extend(block)

# Trim or pad if n_trials is not divisible by 5
trial_type = trial_type_blocks[:n_trials]

# Initialize lists for data storage
reaction_times = []
stop_signal_delays = []
accuracies = []

# Initial SSD
ssd = ssd_start

for i, trial in enumerate(trial_type):
    # Accuracy
    acc = np.random.binomial(1,
                             np.clip(np.random.normal(params[trial]['accuracy']['mean'],
                                                      params[trial]['accuracy']['std']), 0, 1))
    accuracies.append(acc)

    # Stop Signal Delay adjustment
    if trial == 2:
        if acc == 1:
            ssd = min(ssd + ssd_increment, ssd_max)  # Increase by increment, but cap at ssd_max
        else:
            ssd = max(ssd - ssd_increment, ssd_min)  # Decrease by increment, but not below ssd_min
    stop_signal_delays.append(ssd)

    # Reaction Time (timeseries) based on accuracy and trial type
    if trial == 0:
        if acc == 1:
            rt = 1.25
        else:
            rt = np.random.uniform(ssd, 1.25)
    elif trial == 1:
        if acc == 1:
            rt = np.random.uniform(ssd, 1.25)
        else:
            rt = np.random.choice([np.random.uniform(0, ssd), 1.25])
    else:  # trial == 2
        if acc == 1:
            rt = np.random.uniform(ssd, 1.25)
        else:
            rt = np.random.choice([np.random.uniform(0, ssd), 1.25])

    # Ensure reaction time is within 0 to 1.5 seconds
    rt = np.clip(rt, 0, 1.5)
    reaction_times.append(rt)

# Create DataFrame
data = pd.DataFrame({
    'trial_type': trial_type,
    'reaction_time': reaction_times,
    'stop_signal_delay': stop_signal_delays,
    'accuracy': accuracies
})

# Display first few rows to check data
print(data.head())

# Optionally, check statistics by trial type
for t in [0, 1, 2]:
    print(f"\nStatistics for Trial Type {t}:")
    print(data[data['trial_type'] == t].describe())

   trial_type  reaction_time  stop_signal_delay  accuracy
0           1       0.485481               0.30         1
1           2       0.602695               0.35         1
2           1       1.196413               0.35         1
3           1       0.320203               0.35         0
4           0       1.250000               0.35         1

Statistics for Trial Type 0:
       trial_type  reaction_time  stop_signal_delay   accuracy
count        90.0      90.000000          90.000000  90.000000
mean          0.0       1.010181           0.216667   0.566667
std           0.0       0.339705           0.112928   0.498312
min           0.0       0.074085           0.050000   0.000000
25%           0.0       0.766238           0.100000   0.000000
50%           0.0       1.250000           0.250000   1.000000
75%           0.0       1.250000           0.300000   1.000000
max           0.0       1.250000           0.400000   1.000000

Statistics for Trial Type 1:
       trial_type  reacti

In [19]:
import numpy as np
import pandas as pd

# Seed for reproducibility
np.random.seed(42)

n_trials = 450

# Define parameters for each trial type
params = {
    0: {
        'accuracy': {'mean': 0.533, 'std': 0.501},
        'reaction_time': {'mean': 1.038, 'std': 0.522}
    },
    1: {
        'accuracy': {'mean': 0.859, 'std': 0.348},
        'reaction_time': {'mean': 0.844, 'std': 0.366}
    },
    2: {
        'accuracy': {'mean': 0.3555, 'std': 0.4813},
        'reaction_time': {'mean': 0.581432, 'std': 0.4011}
    }
}

# Define SSD (Stop Signal Delay) constraints
ssd_start = 0.300
ssd_min = 0.050
ssd_max = 1.100
ssd_increment = 0.050

# Generate trial types in blocks of 5 trials with random order
trial_type_blocks = []
for _ in range(n_trials // 5):
    block = np.random.permutation([1, 1, 1, 0, 2])
    trial_type_blocks.extend(block)

# Trim or pad if n_trials is not divisible by 5
trial_type = trial_type_blocks[:n_trials]

# Initialize lists for data storage
reaction_times = []
stop_signal_delays = []
accuracies = []

# Initial SSD
ssd = ssd_start

for i, trial in enumerate(trial_type):
    # Accuracy
    acc = np.random.binomial(1,
                             np.clip(np.random.normal(params[trial]['accuracy']['mean'],
                                                      params[trial]['accuracy']['std']), 0, 1))
    accuracies.append(acc)

    # Reaction Time based on trial type and accuracy
    if trial == 0:
        if acc == 1:
            rt = 1.25
        else:
            rt = np.random.uniform(ssd, 1.25)
    elif trial == 1:
        if acc == 1:
            rt = np.random.uniform(ssd, 1.25)
        else:
            rt = np.random.choice([np.random.uniform(0, ssd), 1.25])
    else:  # trial == 2
        if acc == 1:
            rt = np.random.uniform(ssd, 1.25)
        else:
            rt = np.random.choice([np.random.uniform(0, ssd), 1.25])

    reaction_times.append(rt)

    # Stop Signal Delay adjustment for next trial if current trial is type 2
    if trial == 2:
        if acc == 1:
            ssd = min(ssd + ssd_increment, ssd_max)
        else:
            ssd = max(ssd - ssd_increment, ssd_min)

    stop_signal_delays.append(ssd)

# Create DataFrame
data = pd.DataFrame({
    'trial_type': trial_type,
    'reaction_time': reaction_times,
    'stop_signal_delay': stop_signal_delays,
    'accuracy': accuracies
})

# Display first few rows to check data
print(data.head())

# Optionally, check statistics by trial type
for t in [0, 1, 2]:
    print(f"\nStatistics for Trial Type {t}:")
    print(data[data['trial_type'] == t].describe())

   trial_type  reaction_time  stop_signal_delay  accuracy
0           1       0.485481               0.30         1
1           2       0.566734               0.35         1
2           1       1.196413               0.35         1
3           1       0.320203               0.35         0
4           0       1.250000               0.35         1

Statistics for Trial Type 0:
       trial_type  reaction_time  stop_signal_delay   accuracy
count        90.0      90.000000          90.000000  90.000000
mean          0.0       1.010181           0.216667   0.566667
std           0.0       0.339705           0.112928   0.498312
min           0.0       0.074085           0.050000   0.000000
25%           0.0       0.766238           0.100000   0.000000
50%           0.0       1.250000           0.250000   1.000000
75%           0.0       1.250000           0.300000   1.000000
max           0.0       1.250000           0.400000   1.000000

Statistics for Trial Type 1:
       trial_type  reacti

In [20]:
data


,trial_type,reaction_time,stop_signal_delay,accuracy
0,1,0.485481,0.30,1
1,2,0.566734,0.35,1
2,1,1.196413,0.35,1
3,1,0.320203,0.35,0
4,0,1.250000,0.35,1
...,...,...,...,...
445,0,1.250000,0.20,1
446,1,0.382440,0.20,1
447,1,1.250000,0.20,0
448,2,1.250000,0.15,0
